### 2-2. Assignment(week6)

**[before]** charcater-level RNN

appl → pple

**[Assigment]** word-level RNN

Repeat is the best medicine for → is the best medicine for memory

- Hint: one-hot vector 대신 embedding 사용


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()
vocab = list(set(sentence))
print(vocab)

['Repeat', 'for', 'medicine', 'the', 'memory', 'best', 'is']


In [3]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}
word2index['<unk>']=0

index2word = {value: key for key, value in word2index.items()}

print(index2word)
print(word2index)

{1: 'Repeat', 2: 'for', 3: 'medicine', 4: 'the', 5: 'memory', 6: 'best', 7: 'is', 0: '<unk>'}
{'Repeat': 1, 'for': 2, 'medicine': 3, 'the': 4, 'memory': 5, 'best': 6, 'is': 7, '<unk>': 0}


In [4]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] 
    input_seq, label_seq = encoded[:-1], encoded[1:] 
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) 
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) 
    
    return input_seq, label_seq

X, Y = build_data(sentence, word2index)

print(X)
print(Y)

tensor([[1, 7, 4, 6, 3, 2]])
tensor([[7, 4, 6, 3, 2, 5]])


In [5]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        # print(x.size())
        output = self.embedding_layer(x)
        # print(output.size())
        output, hidden = self.rnn_layer(output)
        # print(output.size())
        output = self.linear(output)
        # print(output.size())
        return output.view(-1, output.size(2))

In [6]:
vocab_size = len(word2index)
input_size = 5
hidden_size = 20

model = Net(vocab_size, input_size, hidden_size, batch_first=True)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters())

In [7]:
decode = lambda y: [index2word.get(x) for x in y]

print(decode)

for step in range(200):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_function(output, Y.view(-1))
    loss.backward()
    optimizer.step()
    
    if step % 10 == 0:
        print("[{:02d}/200] {:.4f} ".format(step + 1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

<function <lambda> at 0x0000025B95486F70>
[01/200] 2.1398 
Repeat memory memory for for Repeat for

[11/200] 2.0062 
Repeat for memory for for for for

[21/200] 1.8762 
Repeat is memory for for for for

[31/200] 1.7432 
Repeat is the best for for memory

[41/200] 1.6014 
Repeat is the best for for memory

[51/200] 1.4472 
Repeat is the best for for memory

[61/200] 1.2807 
Repeat is the best medicine for memory

[71/200] 1.1079 
Repeat is the best medicine for memory

[81/200] 0.9403 
Repeat is the best medicine for memory

[91/200] 0.7886 
Repeat is the best medicine for memory

[101/200] 0.6581 
Repeat is the best medicine for memory

[111/200] 0.5489 
Repeat is the best medicine for memory

[121/200] 0.4588 
Repeat is the best medicine for memory

[131/200] 0.3851 
Repeat is the best medicine for memory

[141/200] 0.3252 
Repeat is the best medicine for memory

[151/200] 0.2766 
Repeat is the best medicine for memory

[161/200] 0.2373 
Repeat is the best medicine for memory

[171/20